#Pattern identification tool

Below are two modules that are useful together as a tool for the identification of graphical patterns, given from a module trained in the recognition of the main patterns of the financial market. On the one hand, there is a module that counts the number of patterns of a market in a specific period of time, in order to know the presence of a certain type of graphical pattern in a market. On the other hand, there is a module capable of recognizing in real time the presence of the requested pattern, for the latter makes use of an API.

---

In [7]:
#Dependencies
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from pprint import PrettyPrinter

#Library for visualization of data on candlestick charts
!pip install mpl_finance
import mpl_finance as mpf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#Connection where the data and the trained model are located.

from google.colab import drive
drive.mount('/content/drive')
model_path = '/content/drive/MyDrive/Tesis de grado/CNN_model_pattern.h5'
model = keras.models.load_model(model_path)

Mounted at /content/drive


In [3]:
# Extra functions

window = 28
# GAF function, converts a time series to a Gramian matrix.
def serie_tiempo_gaf(serie,valueMax,valueMin):
  if valueMax == valueMin:
      mGaf = np.zeros((len(serie), len(serie)))
  else:
      serieNorm = np.array((serie-valueMin) / (valueMax-valueMin))
      serieNormArc = np.arccos(serieNorm)
      mGaf = np.zeros((len(serieNorm), len(serieNorm)))
      for f in range(len(serieNorm)):
          for c in range(len(serieNorm)):
              mGaf[f, c] = np.cos(serieNormArc[f] + serieNormArc[c])
  return mGaf


# Function that receives a set of patterns with a predefined structure and that are converted through the GAF method.
def create_gaf(serie_pattern):
  data = serie_pattern.copy()
  gaf = np.zeros((data.shape[0], data.shape[1], data.shape[1], data.shape[2]))
  for i in range(data.shape[0]):
      for c in range(data.shape[2]):
          oneCat = data[i, :, c]
          maxV = np.amax(oneCat)
          minV = np.amin(oneCat)
          result = serie_tiempo_gaf(oneCat, maxV, minV)
          gaf[i, :, :, c] = result
  return gaf


# Function that returns a structured data series with the candlestick data for each pattern.
def create_serie(df_pattern, max='max', min='min', open='open', close='close'):
  total = int(df_pattern.shape[0] - window )
  if total == 0 and df_pattern.shape[0] == window:
    total = 1
  serie = np.zeros((total, window, 4))
  for N in range(total):
    df = df_pattern.loc[(df_pattern.index >= (N)) & (df_pattern.index < (N + window))]
    serie[N, :, 0] = df[open]
    serie[N, :, 1] = df[max]
    serie[N, :, 2] = df[min]
    serie[N, :, 3] = df[close]
  return serie

# function to view candlestick chart of labeled pattern
def pattern_show_image(signal, target, max='max', min='min', open='open', close='close'):           
  fontsize=12
  plt.rcParams['xtick.labelsize'] = fontsize  
  plt.rcParams['ytick.labelsize'] = fontsize 
  plt.rcParams['axes.titlesize'] = fontsize           
  fig = plt.figure(figsize=(8, 8))
  ax = plt.subplot2grid((1, 1), (0, 0))           
  ax.set_xticks(range(10))
  ax.set_xticklabels(target.index)                     
  arr = np.c_[range(target.shape[0]), target[[open, max, min, close]].values]
  mpf.candlestick_ohlc(ax, arr, width=0.5, alpha=1, colordown='#ff1717', colorup='#53c156')          
  locs, labels = plt.xticks() 
  plt.setp(labels , rotation = 45)
  plt.grid()
  ax.legend(loc = 'best', prop = {'size': fontsize})
  title_name = signal
  ax.set_title(title_name)
  fig.subplots_adjust(bottom = 0.25)       
  name = signal
  plt.axis('off')
  plt.show()



---

# Search by market
Module to run the model trained on a data set of a specific market, in order to count the number of certain patterns that have more presence in a specific market.

In [17]:
# Test data
r = create_gaf(create_serie(pd.read_csv('/content/drive/MyDrive/Tesis de grado/datos/Procesados/Obtenidos/S&H/patron_cabezaHombros_23_23_EURUSD.csv')))

# Use of the trained model to evaluate the data
prediction = model.predict(r)

# Detection accuracy 
accuracy_data = 0.95

# Configuration of the pattern to be identified
pattern_to_detect = 's&h'
types_pattern = {
    's&h': 0,
    'dt': 1
}
position_serie = types_pattern[pattern_to_detect]

aux = 0
for item in prediction:
  if item[position_serie] > accuracy_data:
    aux = aux + 1

# Number of patterns found in the data set
print('Patterns detect: ', aux)


Patterns detect:  35


---

#Real-time pattern identification
Module to identify a certain pattern chosen within a market. For this it communicates through an API with the data of a particular market and every minute with the new information about the formation of a new Japanese candlestick is analyzed for the possible presence of a pattern.

In [7]:
import time
from datetime import datetime
from datetime import timedelta
import requests

# Configuration variables on the API 
currency = "EURUSD"
api_key = "*************"
time_str = '01-09-2021 11:12'
date_format_str = '%d-%m-%Y %H:%M'

given_time = datetime.strptime(time_str, date_format_str)
accuracy_pattern = 0.95
pattern_size = 28
aux_windows_pattern = []

# Type of pattern to be identified
pattern_to_detect = 's&h'
types_pattern = {
    's&h': 0,
    'dt': 1
}
position_serie = types_pattern[pattern_to_detect]

# Market data are processed in this function, provided that there is a previous buffer of data to contain a pattern.
# When the information is processed, the market data is transformed by GAF, then it goes through the trained model.
# If a pattern is identified above the agreed upon presicion, it is reported and displayed on the screen.
def build_data_predict(data):
  if len(aux_windows_pattern) == pattern_size:
    df = pd.DataFrame(aux_windows_pattern)
    prediction = model.predict(create_gaf(create_serie(df,'high','low','open','close')))
    if prediction[0][position_serie] > accuracy_pattern:
      print('Prediction of ',prediction[0][position_serie])
      pattern_show_image('signal', df, 'high','low','open','close')
    else:
      print('Pattern not detect. Prediction of ',prediction[0][position_serie])
    aux_windows_pattern.append(data)
    aux_windows_pattern.pop(0)
  else:
    aux_windows_pattern.append(data)


# Time loop, advancing one minute at a time and calling the API and entering the obtained data to a function.
while True:
    given_time = given_time + timedelta(minutes=1)
    aux_date = str(given_time).replace(" ", "-")
    url = "https://marketdata.tradermade.com/api/v1/minute_historical?currency="+str(currency)+"&date_time="+str(aux_date[0:16])+"&api_key="+str(api_key)
    response = requests.get(url)
    build_data_predict(response.json())
    time.sleep(60)






Prediction of  0.07244052
Prediction of  0.07244052
Prediction of  0.07244052
Prediction of  0.07244052
Prediction of  0.07244052
Prediction of  0.07244052
Prediction of  0.07244052
Prediction of  0.07244052
Prediction of  0.07244052


KeyboardInterrupt: ignored